In [258]:
############################# DEMO ####################

data = np.array([[12345678,'ABC'],[23456789,'DEF'],[34567890,'GHI'],[45678901,'JKL'],[56789012, 'MNO'],
                 [12345678,'BOO'],[12341008,'CDD'],[23456789,'DEF'],[23456789,'XCD'],[23456789,'IOP']
                 ,[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP']
                 ,[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP']
                 ,[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP']
                 ,[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP'],[23456789,'IOP']
                ])



In [3]:
################### IMPORT LIBRARIES ################
import numpy as np
import pandas as pd
import spacy

In [12]:
########### EXTRACT DATA ###########
df_dataset = pd.read_excel('1-converted.xlsx')
df_plbook = pd.read_excel('pl_numbers.xlsx')
sheet = df_dataset.values
data = sheet


In [5]:
# Part I

from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
# import enchant

In [6]:
def removeAbbreviations(statement):
    
    abbr = dict()

    abbr["ft"] = "foot"
    abbr["mm"] = "millimeter"
    abbr["cm"] = "centimeter"
    abbr["m"] = "meter"
    abbr["km"] = "kilometer"
    abbr["yd"] = "yard"
    abbr["mi"] = "mile"
    abbr["lb"] = "pound"
    abbr["oz"] = "ounce"
    abbr["mg"] = "milligram"
    abbr["g"] = "gram"
    abbr["kg"] = "kilogram"
    abbr["ml"] = "milliliter"
    abbr["l"] = "liter"
    abbr["mph"] = "miles per hour"
    abbr["kmph"] = "kilometers per hour"
    abbr["cu"] = "cubic"
    abbr["rpm"] = "revolutions per minute"
    abbr["&"] = "and"
    abbr["$"] = "dollar"
    abbr["°"] = "degree"
    abbr['"'] = "inch"
    
    statement_ = ""
    
    for word in statement.split():
        if word in abbr:
            statement_ = statement_ + " " + abbr[word]
        else:
            statement_ = statement_ + " " + word
            
    return statement_

In [7]:
def spelling(spellCheck, tokens1, tokens2):
    
    spell = SpellChecker()
    misspelled = spell.unknown(spellCheck)

    spellCheck_ = []
    for i in misspelled:
        spellCheck_.append(i)

    for i in range(len(spellCheck_)):
        spellCheck_[i] = spell.correction(spellCheck_[i])
    
    for word in spellCheck_:
        if not wordnet.synsets(word):
            spellCheck_.remove(word)
        
    for word in spellCheck_:
        flag1 = True
        flag2 = True
        for i in tokens1:
            if word == i:
                flag1 = False
                break
        for i in tokens2:
            if word == i:
                flag2 = False
                break
        if flag1 == True and flag2 == True:
            spellCheck_.remove(word)
    
    return spellCheck_

In [8]:
def checkSimilar(list1, list2):
    list1_copy = []
    for i in list1:
        list1_copy.append(i)
    list2_copy = []
    for i in list2:
        list2_copy.append(i)
    
    dict1 = dict()
    for i in list1:
        dict1[i] = 0
    for i in list2:
        dict1[i] = 0
    for i in list1:
        dict1[i] = 1 + dict1[i]
        
    dict2 = dict()
    for i in list1:
        dict2[i] = 0
    for i in list2:
        dict2[i] = 0
    for i in list2:
        dict2[i] = 1 + dict2[i]
        
    flag = True    
    for i in list1:
        if dict1[i] != dict2[i]:
            flag = False
            break
            
    return flag

In [9]:
def checkSimilarity(S1, S2):
    
    S1 = S1.lower()
    S2 = S2.lower()
    
    S1 = removeAbbreviations(S1)
    S2 = removeAbbreviations(S2)
    
    S1 = S1[1:len(S1)]
    S2 = S2[1:len(S2)]
    
    tokenizer = RegexpTokenizer(r'\w+')
    words1 = tokenizer.tokenize(S1)
    words2 = tokenizer.tokenize(S2)
        
    spellCheck = []
    for i in words1:
        flag = True
        for j in words2:
            if i == j:
                flag = False
                break
        if flag == True:
            spellCheck.append(i)
    for i in words2:
        flag = True
        for j in words1:
            if i == j:
                flag = False
                break
        if flag == True:
            spellCheck.append(i)
        
    corrected_spellings = spelling(spellCheck, words1, words2)
    
    for i in corrected_spellings:
        if i in spellCheck:
            spellCheck.remove(i)
    
    for i in range(len(corrected_spellings)):
        for j in range(len(words1)):
            if words1[j] == spellCheck[i]:
                words1[j] = corrected_spellings[i]
        for j in range(len(words2)):
            if words2[j] == spellCheck[i]:
                words2[j] = corrected_spellings[i]

    porter = PorterStemmer()

    lemmatize1 = []
    lemmatize2 = []

    for word in words1:
        lemmatize1.append(porter.stem(word))
    for word in words2:
        lemmatize2.append(porter.stem(word))
        
    for i in range(len(lemmatize1)):
        words1[i] = lemmatize1[i]
    for i in range(len(lemmatize2)):
        words2 = lemmatize2
    
    tokens1 = []
    for i in words1:
        tokens1.append(i)
    tokens2 = []
    for i in words2:
        tokens2.append(i)
    
    d = enchant.Dict("en_US")
    for word in words1:
        if d.check(word) == False:
            words1.remove(word)
    for word in words2:
        if d.check(word) == False:
            words2.remove(word)
            
    technical_terms1 = []
    for i in tokens1:
        check = False
        for j in words1:
            if i == j:
                check = True
                break
        if check == False:
            technical_terms1.append(i)

    technical_terms2 = []
    for i in tokens2:
        check = False
        for j in words2:
            if i == j:
                check = True
                break
        if check == False:
            technical_terms2.append(i)
            
    flag = False
    flag1 = True

    if len(technical_terms1) == len(technical_terms2) and len(technical_terms1) == 0:
        None
    elif len(technical_terms1) != len(technical_terms2):
        flag1 = False
    else:
        if checkSimilar(technical_terms1, technical_terms2):
            flag1 = True     

    if flag1 == True:
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('english'))
        stop_words1 = [w for w in words1 if not w in stop_words]
        stop_words2 = [w for w in words2 if not w in stop_words]

        flag = True
        if len(stop_words1) != len(stop_words2):
            flag = False
        else:
            flag = checkSimilar(stop_words1, stop_words2)

    return flag


In [10]:
def sameDescription(data1, data2):
    flag = checkSimilarity(data1,data2)
    return flag

In [13]:
############ Driver function for part 1 gives list ############
flag = np.array([False for i in range(data.shape[0])])
flag = flag.reshape((data.shape[0],1))
data = np.c_[data,flag]
n = data.shape[0]
uniqueDescriptions = []
for i in range(n):
    if(data[i][2]=='True'):
        continue
    data[i][2] = 'True'
    row = [data[i][1]]
    for j in range(i+1,n):
        if(data[j][2]=='True'):
            continue
        check = sameDescription(data[i][1],data[j][1])
        if(check==True):
            row.append(data[j][1])
            data[j][2]= 'True'
    uniqueDescriptions.append(row)
    
    
######################## PART 1 ENDS ########################

NameError: name 'enchant' is not defined

In [ ]:
#################### PART 2 BEGINS ########################

data = sheet
df = df_plbook
df['desc']=df['Description_A(1)']+" "+df['Description_A(2)']+" "+df['Description_B']# Load English tokenizer, tagger, parser, NER and word vectors


lis=df['desc'].tolist()
nlp = spacy.load('en_core_web_sm')
t =""
for i in range(data.shape[0]):
    t+= str(lis[i])
    t+=" "
doc = nlp(t)



def group(d):
    m_val = 0
    m_index = 0
    des_str = nlp(u""+d)

    for i in range(308):
#     print(i)
        db_str = nlp(str(lis[i]))
        similarity = des_str.similarity(db_str)
#     print(i,db_str,similarity)
        if(similarity>m_val):
            m_index = i
            m_val=similarity
#     print("max Similarity",m_val)
    a = df.iloc[m_index,1]
    b = df.iloc[m_index,2]
    print("A",a,"B",b)
    x=df.iloc[m_index,0];
    x=str("{:04d}".format(x))
    
    return int(x);


In [ ]:
############# DRIVER FOR PART 2 ################

mapping = {}
for element in uniqueDescriptions:
    pl = group(element[0])
    if pl not in mapping:
        mapping[pl] = [element]
    else:
        mapping[pl].append(element)


In [ ]:
################ PART 2 ENDS ####################

In [ ]:
############### PART 3 BEGINS ##################
def calc(A,B,C):
    pl = int(A+B+C)
    sum =0
    i = 2
    while(pl!=0):
        
        rem = pl%10
        sum+=rem*i
        pl//=10
        i+=1
    if(sum%11 == 10):
        return '0'
    return str(sum%11)

In [ ]:
def ChangeHex(n):
    if (n<=1):
        return (str(n))
    else:
        ans = ChangeHex( n // 16 )
        x =(n%16)
        s = str(x)
        if (x < 10):
            return (ans+s)  
        if (x == 10):
            return (ans+"A")
        if (x == 11):
            return (ans+"B")
        if (x == 12):
            return (ans+"C")
        if (x == 13):
            return (ans+"D")
        if (x == 14):
            return (ans+"E")
        if (x == 15):
            return (ans+"F")
        else:
            return (5)

In [ ]:
def fix(mapping, pl, descriptions):
    
    pl = str(pl)
    key = 0
    if(len(pl)<4):
        pl = '0'+pl
    A = pl[0:2]
    B = pl[2:4]
    
    for i in range(len(descriptions)):
        hex = ChangeHex(key)
        if(len(hex)<2):
            hex = '00'+hex
        elif(len(hex)<3):
            hex = '0'+hex
            
        allotedPl = A+B+hex+D#calc(A,B,key)
        mapping[allotedPl] = [dsc]
        key+=1
    
    mapping.pop(pl,None)  
    
    
    return mapping   

In [ ]:
############### DRIVER FOR PART 3 ##############
pl = sorted(mapping.keys())

for num in pl:
    val = mapping[num]
    if(len(val)>1):
        print('\033[30;1;46m',num,':',mapping[num])
    else:    
        print('\033[30;1;m',num,':',mapping[num])

In [ ]:
for num in pl:
    val = mapping[num]
    if(len(val)>1):
        mapping = fix(mapping,num,val)
        
print('\n\n\n###########\n\n')
for key in mapping:
    print(key,':', mapping[key])